In [31]:
from math import sin, cos
from functools import partial
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import cv2
import csv

import sys
sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc

In [101]:
# Load obstacle data

data = np.genfromtxt('/home/oscar_palfelt/MSc_thesis/LearnedSamplingDistributions/narrowDataFile.txt', delimiter=',', usecols=range(27))
numEntries = data.shape[0]

obsData = data[:, 6:-13]

xyInitData = data[:,-12:-10]
yawInitData = yawInitData = np.repeat([[-np.pi/2]], numEntries, axis=0)
initData = np.concatenate((xyInitData, yawInitData), axis=1)

xyGoalData = data[:,-6:-4]
yawGoalData = np.repeat([[-np.pi/2]], numEntries, axis=0)
goalData = np.concatenate((xyGoalData, yawGoalData), axis=1)


In [33]:
# Generate occupancy grid
def isSampleFree(sample, obs):
    for o in list(range(0,obs.shape[0]//(2*dimW))): # python 2 -> 3: use list(), use //
        isFree = 0
        for d in range(0,sample.shape[0]):
            if (sample[d] < obs[2*dimW*o + d] or sample[d] > obs[2*dimW*o + d + dimW]):
                isFree = 1
                break
        if isFree == 0:
            return 0
    return 1

gridSize = 11

# process data into occupancy grid
occGrids = np.zeros([numEntries,gridSize*gridSize])
occGridSamples = np.zeros([gridSize*gridSize, 2])
gridPointsRange = np.linspace(0,1,num=gridSize)

idx = 0;
for i in gridPointsRange:
    for j in gridPointsRange:
        occGridSamples[idx,0] = i
        occGridSamples[idx,1] = j
        idx += 1;

for j in range(0,numEntries,1):
    dw = 0.1
    dimW = 3
    gap1 = obsData[j,0:3]
    gap2 = obsData[j,3:6]
    gap3 = obsData[j,6:9]

    obs1 = [0, gap1[1]-dw, -0.5,          gap1[0], gap1[1], 1.5]
    obs2 = [gap2[0]-dw, 0, -0.5,          gap2[0], gap2[1], 1.5]
    obs3 = [gap2[0]-dw, gap2[1]+dw, -0.5, gap2[0], 1, 1.5]
    obs4 = [gap1[0]+dw, gap1[1]-dw, -0.5, gap3[0], gap1[1], 1.5]
    obs5 = [gap3[0]+dw, gap1[1]-dw, -0.5, 1, gap1[1], 1.5]
    obs = np.concatenate((obs1, obs2, obs3, obs4, obs5), axis=0)
    
    if j % 5000 == 0:
        print('Iter: {}'.format(j))
        
    occGrid = np.zeros(gridSize*gridSize)
    for i in range(0,gridSize*gridSize):
        occGrid[i] = isSampleFree(occGridSamples[i,:],obs)
    occGrids[j,:] = occGrid

Iter: 0
Iter: 5000
Iter: 10000
Iter: 15000
Iter: 20000
Iter: 25000
Iter: 30000
Iter: 35000
Iter: 40000
Iter: 45000
Iter: 50000
Iter: 55000
Iter: 60000
Iter: 65000
Iter: 70000
Iter: 75000
Iter: 80000


In [301]:
# define planning problem

class MyDecomposition(oc.GridDecomposition):
    def __init__(self, length, bounds):
        super(MyDecomposition, self).__init__(length, 2, bounds)
    def project(self, s, coord):
        coord[0] = s.getX()
        coord[1] = s.getY()
    def sampleFullState(self, sampler, coord, s):
        sampler.sampleUniform(s)
        s.setXY(coord[0], coord[1])
## @endcond

def isStateValid(occGrid, spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied

    u = round(state.getX() * (occGrid.shape[1] - 1)) # right pointing image axis
    v = round((occGrid.shape[0] - 1) * (1 - state.getY())) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return occGrid[v,u] > 0
    else:
        return False

def propagate(start, control, duration, state):
    v = 0.1
    state.setX(start.getX() + v * duration * cos(start.getYaw()))
    state.setY(start.getY() + v * duration * sin(start.getYaw()))
    state.setYaw(start.getYaw() + control[0] * duration)


def problemDef(occGrid, initState, goalState):
    # construct the state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.0)
    bounds.setHigh(1.0)
    space.setBounds(bounds)

    # create a control space
    cspace = oc.RealVectorControlSpace(space, 1)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(1)
    cbounds.setLow(-1.3)
    cbounds.setHigh(1.3)
    cspace.setBounds(cbounds)

    # define a simple setup class
    ss = oc.SimpleSetup(cspace)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, occGrid, ss.getSpaceInformation())))
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))

    # set planner
    si = ss.getSpaceInformation()
    # SyclopEST and SyclopRRT require a decomposition to guide the search
    decomp = MyDecomposition(32, bounds)
    planner = oc.SyclopEST(si, decomp)
    #planner = oc.SyclopRRT(si, decomp)
    ss.setPlanner(planner)
    # (optionally) set propagation step size
    si.setPropagationStepSize(.2)

    return ss


def plan(planObj, initState, goalState):

    space = ob.SE2StateSpace()

    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.05)

    solved = planObj.solve(0.5)

    if solved:
        return np.loadtxt(StringIO(planObj.getSolutionPath().printAsMatrix()))

In [302]:
# generate planning data

# planera endast för det unika mapsen

trajSamples = np.zeros(shape=(numEntries,3)) # x, y, yaw

planObj = problemDef(np.rot90(occGrids[i].reshape(gridSize, gridSize)), initData[i], goalData[i])

for i in range(8,20):
    sol = plan(planObj, initState=initData[i], goalState=goalData[i])
    if not sol is None:

        rndInd = np.random.randint(0, sol.shape[0] - 1) # skippa ~första 2 / sista 2 elements?

        rndSample = sol[rndInd, 0:3]

        trajSamples[i,:] = rndSample

    planObj.clear()
 
    if i % 1 == 0:
        print('\nIter: {}\n'.format(i))

np.savetxt('samplePaths', trajSamples, delimiter=',',)


         at line 77 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/src/SpaceInformation.cpp


Info:    SyclopEST: Starting planning with 1 states already in datastructure

Iter: 8
Info:    ProblemDefinition: Adding approximate solution from planner SyclopEST

Info:    Solution found in 0.500020 seconds
Info:    SyclopEST: Starting planning with 1 states already in datastructure

Iter: 9
Info:    Solution found in 0.398116 seconds

Info:    SyclopEST: Starting planning with 1 states already in datastructure

Iter: 10
Info:    Solution found in 0.462104 seconds

Info:    SyclopEST: Starting planning with 1 states already in datastructure

Iter: 11
Info:    ProblemDefinition: Adding approximate solution from planner SyclopEST

Info:    Solution found in 0.500109 seconds
Info:    SyclopEST: Starting planning with 1 states already in datastructure
Iter: 12


Info:    Solution found in 0.350082 seconds
Info:    SyclopEST: Starting planning with 1 states already in datastructure

Iter: 13
Info:    ProblemDefinition: Adding approximate solution from planner SyclopEST

Info:    Solution